In [3]:
import pandas as pd
import yaml

customer_statistics = '/content/sample_data/customer_statistics.csv'
orders = '/content/sample_data/orders.csv'
customer_demographics = '/content/sample_data/customer_demographics.yaml'


with open(customer_demographics, 'r') as file:
    data = yaml.safe_load(file)

df1 = pd.DataFrame(data).transpose()


In [5]:
df1

,address,city,credit_card_expires,credit_card_number,credit_card_provider,credit_card_security_code,customer_id,email,name,phone_number,state,zip_code
0,621 Chavez Ramp Suite 938,West Deniseport,06/31,2269172571677458,Diners Club / Carte Blanche,338,a8c966a0-b670-4044-826c-0e6bfe708259,brett00@example.com,Ronald Rodgers,None,New York,59237
1,592 Mccoy River,Port Dannymouth,11/24,676320507830,JCB 16 digit,246,56b850e0-9320-4aa0-8cd4-d1199903c90e,smithjennifer@example.net,Mary Mitchell,None,Montana,17401
2,8923 Aaron Turnpike,Port Rhondaton,11/31,4607779976715144588,JCB 16 digit,869,5ec9f613-adb1-4e69-8a1a-9530cb777886,twhite@example.net,Nathan Wells,+1-893-568-2182x593,North Dakota,07166
3,496 Margaret Shoals,Smithborough,04/33,4234465830748237,Mastercard,497,ca182662-b281-433a-a82b-8e061f7bf2cd,nguyenjennifer@example.com,Kelly Ramos,820-824-2003,Iowa,42755
4,0807 Logan Fords Suite 355,Contrerasside,10/25,4985681645741,Discover,927,1e7c3a60-4788-4711-809f-b6f8bfbdb18a,michaelabarr@example.net,Rebekah Fitzgerald,+1-269-384-1623x34240,Connecticut,64282
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,419 Lopez Crossing,North Nicolechester,01/30,639083591672,VISA 13 digit,079,0707f817-5018-4060-b01c-2c5b53374a2e,andrewrogers@example.net,Tracey Edwards,001-555-466-6738x7224,Alabama,15476
9996,38594 Franklin Pine,Port Joshua,01/31,4197551271949528,VISA 13 digit,237,abc92c1c-788c-4e61-a40d-c6754c99bdc7,sduncan@example.com,Ashley Peterson,(434)761-1066,New Jersey,58035
9997,535 Kimberly Station Apt. 546,Garzaville,07/24,36597049683777,VISA 16 digit,287,418cc301-e111-4080-b8f7-8d9f1d9dc304,rhatfield@example.net,Sandra Baldwin MD,867.829.9891x64114,Connecticut,16016
9998,35316 Becky Terrace,Port Donald,04/24,6595168672629675,JCB 16 digit,113,7caf0251-8040-4c1e-8190-46133fde16ab,andreawilliams@example.org,Karen Green,None,Nevada,97426


In [4]:
df2 = pd.read_csv(customer_statistics)

In [6]:
df2

,customer_id,total_orders,total_items,total_spent
0,8037,5,20,300.39
1,4290,2,3,332.66
2,5367,3,7,643.27
3,4755,10,12,649.12
4,2651,10,9,794.28
...,...,...,...,...
9995,585,10,14,394.88
9996,4604,2,1,858.88
9997,5168,7,8,137.52
9998,3722,5,1,105.75


In [9]:
#clean + merge
clean_df2 = df2.groupby('customer_id').agg({
    'total_orders': 'sum',
    'total_items': 'sum',
    'total_spent': 'sum'}).reset_index()

clean_df2

,customer_id,total_orders,total_items,total_spent
0,0,12,29,2299.91
1,3,7,17,392.78
2,4,17,25,1789.48
3,5,4,21,803.61
4,6,9,39,1935.00
...,...,...,...,...
6278,9993,7,9,405.65
6279,9994,2,10,338.05
6280,9995,1,9,864.16
6281,9996,6,13,649.99
